<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span><ul class="toc-item"><li><span><a href="#Очистка-текста" data-toc-modified-id="Очистка-текста-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Очистка текста</a></span></li><li><span><a href="#Векторизация-обучающей-выборки" data-toc-modified-id="Векторизация-обучающей-выборки-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Векторизация обучающей выборки</a></span></li><li><span><a href="#Векторизация-тестовой-выборки" data-toc-modified-id="Векторизация-тестовой-выборки-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Векторизация тестовой выборки</a></span></li></ul></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li></ul></div>

# Проект для «Викишоп» c BERT
Данный проект выполнялся в colab.research.google.com с включенным GPU

Целью данного проекта - выявление токсичных коментатриев в отзывах на товары магазина.
Нужно создать модель машинного обучения которая будет классифицировать комментарии на позитивные и негативные.
Предоставлен набор данных с разметкой о токсичности правок.

Описание данных:  
Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [10]:
import pandas as pd
import numpy as np
import torch
!pip install transformers
import transformers as ppb # pytorch transformers
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from tqdm import notebook
from sklearn.utils import resample
from sklearn.utils import shuffle
import re
import time
! pip3 install catboost
from catboost import CatBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from numba import cuda
import warnings
warnings.simplefilter('ignore')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [11]:
df = pd.read_csv('https://code.s3.yandex.net/datasets/toxic_comments.csv')

print(df.info(), df.head(), df['toxic'].value_counts(), 'дубликатов', df.duplicated().sum(), sep='\n'*2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159571 non-null  object
 1   toxic   159571 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB
None

                                                text  toxic
0  Explanation\nWhy the edits made under my usern...      0
1  D'aww! He matches this background colour I'm s...      0
2  Hey man, I'm really not trying to edit war. It...      0
3  "\nMore\nI can't make any real suggestions on ...      0
4  You, sir, are my hero. Any chance you remember...      0

0    143346
1     16225
Name: toxic, dtype: int64

дубликатов

0


In [12]:
# Функция даунсемлинга для обучающей выборки

def downsample(features, target, fraction):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_downsampled = pd.concat(
    [features_zeros.sample(frac=fraction, random_state=123)] + [features_ones])
    target_downsampled = pd.concat(
    [target_zeros.sample(frac=fraction, random_state=123)] + [target_ones])
    
    features_downsampled, target_downsampled = shuffle(
    features_downsampled, target_downsampled, random_state=123)
    
    return features_downsampled.reset_index(drop=True), target_downsampled.reset_index(drop=True)


In [13]:
# Выделение в выборках признаков и целевого признака
features = df['text']
target = df['toxic']

# Разбивка на обучающую выборку и тестовую выборку
features_train_all, features_test, target_train_all, target_test = train_test_split(
    features, target, test_size=0.1, random_state=12345)

# Даунсемлинг обучающей выборки
features_train, target_train = downsample(features_train_all, target_train_all, 0.8)
features_test.reset_index(drop=True, inplace=True)
target_train.reset_index(drop=True, inplace=True)

print('Размер обучающей выборки', len(features_train), len(target_train),
                                      '\n', target_train.value_counts())
print()
print('Размер тестовой выборки', len(features_test), len(target_test),
                                      '\n', target_test.value_counts())

Размер обучающей выборки 117802 117802 
 0    103243
1     14559
Name: toxic, dtype: int64

Размер тестовой выборки 15958 15958 
 0    14292
1     1666
Name: toxic, dtype: int64


### Очистка текста

In [ ]:
# Удаление из текста всех символов кроме латинских
features_train = features_train.apply(lambda x: ' '.join(re.sub(r'[^a-zA-Z.,!?]', ' ', x).split()))
features_test = features_test.apply(lambda x: ' '.join(re.sub(r'[^a-zA-Z.,!?]', ' ', x).split()))

### Векторизация обучающей выборки

In [15]:
%%time
# Загрузка обученной модели BERT, токенизатора
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')
# model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/256M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_projector.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


CPU times: user 7.98 s, sys: 1.37 s, total: 9.35 s
Wall time: 17.6 s


In [16]:
# Токенизация предложений из обучающей выборки

max_len = 512 # Макимальная длина списка
tokenized = features_train.apply(
    (lambda x: tokenizer.encode(x, add_special_tokens=True, truncation=True, max_length=max_len)))
padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])
attention_mask = np.where(padded != 0, 1, 0)

In [17]:
# Преобразование предложений в список токенов (обучающая выборка)

batch_size = 200 # Размер батча
embeddings = []

for i in notebook.tqdm(range(-(-padded.shape[0] // batch_size))):
    batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)]).cuda()
    attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)]).cuda()

    with torch.no_grad():
        model.cuda()
        batch_embeddings = model(batch, attention_mask=attention_mask_batch)

    embeddings.append(batch_embeddings[0][:,0,:].cpu().numpy())
    
features_train_fin = np.concatenate(embeddings)
torch.cuda.empty_cache()

  0%|          | 0/590 [00:00<?, ?it/s]

### Векторизация тестовой выборки

In [18]:
# Токенизация предложений из тестовой выборки

tokenized = features_test.apply((lambda x: tokenizer.encode(x, add_special_tokens=True, truncation=True, max_length=max_len)))
padded_t = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])
attention_mask_t = np.where(padded_t != 0, 1, 0)

In [19]:
# Преобразование предложений в список токенов (тестовая выборка)

embeddings = []

for i in notebook.tqdm(range(-(-padded_t.shape[0] // batch_size))):
    batch = torch.LongTensor(padded_t[batch_size*i:batch_size*(i+1)]).cuda()
    attention_mask_batch = torch.LongTensor(attention_mask_t[batch_size*i:batch_size*(i+1)]).cuda()

    with torch.no_grad():
        model.cuda()
        batch_embeddings = model(batch, attention_mask=attention_mask_batch)

    embeddings.append(batch_embeddings[0][:,0,:].cpu().numpy())
    
features_test_fin = np.concatenate(embeddings)
torch.cuda.empty_cache()

  0%|          | 0/80 [00:00<?, ?it/s]

In [20]:
# Сохранение данных

# from google.colab import drive
# drive.mount('/content/drive')
# np.save('/content/drive/My Drive/features_train_fin', np.array(features_train_fin))
# np.save('/content/drive/My Drive/target_train', np.array(target_train))
# np.save('/content/drive/My Drive/features_test_fin', np.array(features_test_fin))
# np.save('/content/drive/My Drive/target_test', np.array(target_test))

Mounted at /content/drive


In [21]:
# Загрузка сохраненных данных для обучения и тестирования моделей

# from google.colab import drive
# drive.mount('/content/drive')
# features_train_fin = np.load('/content/drive/My Drive/features_train_fin.npy')
# target_train = np.load('/content/drive/My Drive/target_train.npy')
# features_test_fin = np.load('/content/drive/My Drive/features_test_fin.npy')
# target_test = np.load('/content/drive/My Drive/target_test.npy')

In [22]:
print('Размер обучающей выборки после векторизации;', len(features_train_fin), len(target_train))
print('Размер тестовой выборки после векторизации:', len(features_test_fin), len(target_test))

Размер обучающей выборки после векторизации; 117802 117802
Размер тестовой выборки после векторизации: 15958 15958


## Обучение

In [24]:
%%time
# Модели на обучение
clf1 = CatBoostClassifier(loss_function='Logloss', eval_metric='F1', logging_level='Silent',
                               max_depth=10, task_type='GPU',  random_state=12345)

clf2 = LogisticRegression(n_jobs=-1, random_state=12345)
clf3 = LinearSVC(random_state=12345)
clf4 = RandomForestClassifier(n_jobs=-1, random_state=12345)
clf5 = KNeighborsClassifier(n_jobs=-1)
clf6 = GaussianNB()

# Обучение моделей и получение метрики "f1" на тестовой выборке
for model, label in zip([clf1, clf2, clf3, clf4, clf5, clf6], 
                      ['CatBoostClassifier', 
                       'LogisticRegression', 
                       'LinearSVC',
                       'RandomForestClassifier',
                       'KNeighborsClassifier',
                       'GaussianNB']):
    
        model.fit(features_train_fin, target_train)
        pred = model.predict(features_test_fin)
        score = f1_score(target_test, pred)
        print(label,'f1 =', '%.2f'% score)

CatBoostClassifier f1 = 0.72
LogisticRegression f1 = 0.75


/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,


LinearSVC f1 = 0.76
RandomForestClassifier f1 = 0.65
KNeighborsClassifier f1 = 0.67
GaussianNB f1 = 0.54
CPU times: user 20min 30s, sys: 59.1 s, total: 21min 30s
Wall time: 16min 55s


In [25]:
# Проверка моделей на вменяемость

pred_0 = target_test * 0
score_0 = f1_score(target_test, pred_0)

pred_1 = target_test * 0 + 1
score_1 = f1_score(target_test, pred_1)

print('Предсказание все положительные отзывы f1 =', '%.2f'% score_0)
print('Предсказание все негативные отзывы f1 =', '%.2f'% score_1)

Предсказание все положительные отзывы f1 = 0.00
Предсказание все негативные отзывы f1 = 0.19


## Выводы

Целью данного проекта является построение модели для классификации комментарев на позитивные и негативные.  
В работе было выполнено следующее: 
Был проведен анализ датасета. В результате проведенного анализа обнаружен дисбаланс классов – негативных коментариев в 9 раз меньше положительных.  
Датасет был разбит на обучающую и тестовую выборки. На тестовой выборке проведен даунсемплинг с сохранением всех негативных коментариев.  
Была проведена векторизация обучающей и тестовой выборки. На обработанных выборках обучены модели: 'CatBoostClassifier', 
'LogisticRegression', 'LinearSVC', 'RandomForestClassifier', 'KNeighborsClassifier', 'GaussianNB'.  
Полученные модели проверены на вменяемость.  
Лучшую метрику качества F1 на тестовых данных показали модели LinearSVC f1 = 0.76 и LogisticRegression f1 = 0.75